# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   692.427321 -0.032362  1234  692.394959  692.459683  1384.789918   
6   598.013695 -0.127190  1234  597.886505  598.140886  1195.773010   
7   177.614574 -0.216988  1234  177.397586  177.831562   354.795172   
8   103.210880 -0.494824  1234  102.716056  103.705704   205.432112   
9   657.791385 -0.559715  1234  657.231669  658.351100  1314.463339   
10  497.276031  0.006771  1234  497.282802  497.269261   994.565604   
11  638.704530  0.692335  1234  639.396865  638.012195  1278.793730   
12  275.596849  0.213174  1234  275.810023  275.383675   551.620046   
13  623.935431 -0.864084  1234  623.071347  624.799515  1246.142694   
14  211.933734  0.219751  1234  212.153486  211.713983   424.306971   

              f  
5   1384.919365  
6   1196.281772  
7    355.663123  
8    207.411408  
9   1316.702200  
10   994.538521  
11  1276.024391  
12   550.767350  
13  1249.599030  
14   423.427967

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -692.394959   692.459683
6   -597.886505   598.140886
7   -177.397586   177.831562
8   -102.716056   103.705704
9   -657.231669   658.351100
10  -497.282802   497.269261
11  -639.396865   638.012195
12  -275.810023   275.383675
13  -623.071347   624.799515
14  -212.153486   211.713983

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -692.394959   692.459683
6   -597.886505   598.140886
7   -177.397586   177.831562
8   -102.716056   103.705704
9   -657.231669   658.351100
10  -497.282802   497.269261
11  -639.396865   638.012195
12  -275.810023   275.383675
13  -623.071347   624.799515
14  -212.153486   211.713983

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-692.39495899,  692.45968254],
       [-597.88650483,  598.14088577],
       [-177.3975861 ,  177.83156171],
       [-102.7160558 ,  103.70570421],
       [-657.23166945,  658.35109982],
       [-497.28280209,  497.26926059],
       [-639.39686487,  638.0121954 ],
       [-275.81002293,  275.38367499],
       [-623.07134722,  624.79951497],
       [-212.15348553,  211.71398333]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1382.789918  -1384.919365
6    1193.773010  -1196.281772
7     352.795172   -355.663123
8     203.432112   -207.411408
9    1312.463339  -1316.702200
10    992.565604   -994.538521
11   1276.793730  -1276.024391
12    549.620046   -550.767350
13   1244.142694  -1249.599030
14    422.306971   -423.427967

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf